In [17]:
import yfinance as yf
import pandas as pd
import numpy as np
import vectorbt as vbt
import empyrical as ep
import warnings
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# Load data
TFEX_S50 = pd.read_excel("S50_TFEX.xlsx")
TFEX_S50['Timestamp'] = pd.to_datetime(TFEX_S50['Timestamp'])
TFEX_S50.set_index('Timestamp', inplace=True)

# Define the window size
window = 30  # Adjust the window size

# Get the column pairs
cols = TFEX_S50.columns
num_cols = len(cols)
pairs = [(cols[i], cols[i+1]) for i in range(0, num_cols - 1, 2)]

for first_col, second_col in pairs:
    print(f"Processing pair: {first_col} and {second_col}")
    
    # Prepare data for the current pair
    P24_H2 = TFEX_S50[[first_col, second_col]].dropna()
    P24_H2[first_col] = pd.to_numeric(P24_H2[first_col], errors='coerce')
    P24_H2[second_col] = pd.to_numeric(P24_H2[second_col], errors='coerce')
    P24_H2['Gap'] = P24_H2[first_col] - P24_H2[second_col]
    P24_H2['Rolling Mean'] = P24_H2['Gap'].rolling(window=window).mean()
    P24_H2['Rolling Std'] = P24_H2['Gap'].rolling(window=window).std()
    P24_H2['Z-score'] = (P24_H2['Gap'] - P24_H2['Rolling Mean']) / P24_H2['Rolling Std']
    P24_H2['EMA'] = P24_H2['Gap'].ewm(span=window, adjust=False).mean()
    
    # fig = make_subplots(
    #     rows=6, cols=1,
    #     shared_xaxes=True,
    #     subplot_titles=(
    #         f'Price of {first_col}', 
    #         f'Price of {second_col}', 
    #         'Gap', 
    #         'Rolling Mean', 
    #         'EMA', 
    #         'Rolling Std and Z-Score'
    #     ),
    #     vertical_spacing=0.1
    # )

    # fig.add_trace(go.Scatter(x=P24_H2.index, y=P24_H2[first_col], mode='lines', name=first_col, line=dict(color='blue')), row=1, col=1)
    # fig.add_trace(go.Scatter(x=P24_H2.index, y=P24_H2[second_col], mode='lines', name=second_col, line=dict(color='orange')), row=2, col=1)
    # fig.add_trace(go.Scatter(x=P24_H2.index, y=P24_H2['Gap'], mode='lines', name='Gap', line=dict(color='purple')), row=3, col=1)
    # fig.add_trace(go.Scatter(x=P24_H2.index, y=P24_H2['Rolling Mean'], mode='lines', name='Rolling Mean', line=dict(color='green')), row=4, col=1)
    # fig.add_trace(go.Scatter(x=P24_H2.index, y=P24_H2['EMA'], mode='lines', name='EMA', line=dict(color='orange', dash='dot')), row=5, col=1)
    # fig.add_trace(go.Scatter(x=P24_H2.index, y=P24_H2['Rolling Std'], mode='lines', name='Rolling Std', line=dict(color='red')), row=6, col=1)
    # fig.add_trace(go.Scatter(x=P24_H2.index, y=P24_H2['Z-score'], mode='lines', name='Z-Score', line=dict(color='cyan')), row=6, col=1)

    # fig.update_layout(
    #     title=f'Prices, Gap, Rolling Statistics, EMA, and Z-Score between {first_col} and {second_col}',
    #     xaxis_title='Date',
    #     yaxis_title='Value',
    #     height=1800,
    #     margin=dict(t=100, b=100, l=80, r=80),
    #     template='plotly_dark'
    # )
    # fig.show()
    
    # Backtest with vectorbt
    entry_long = P24_H2['Z-score'] < -0.5 
    entry_short = P24_H2['Z-score'] > 0.5
    exit_long_short = abs(P24_H2['Z-score']) < 0.01

    portfolio = vbt.Portfolio.from_signals(
        close=P24_H2[first_col],
        entries=entry_long,
        short_entries=entry_short,
        exits=exit_long_short,
        short_exits=exit_long_short
    )

    # Output the performance
    print(f"Performance stats for {first_col} and {second_col}:")
    print(portfolio.stats())
    
    fig = portfolio.plot()
    fig.update_layout(title=f'Trading Strategy Performance for {first_col} and {second_col}', xaxis_title='Date', yaxis_title='Portfolio Value')
    fig.show()


Processing pair: S50H0^2 and S50M0^2
Performance stats for S50H0^2 and S50M0^2:
Start                                         NaT
End                           2020-03-30 00:00:00
Period                                         63
Start Value                                 100.0
End Value                               125.83224
Total Return [%]                         25.83224
Benchmark Return [%]                   -32.068549
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                          6.48393
Max Drawdown Duration                         5.0
Total Trades                                    2
Total Closed Trades                             1
Total Open Trades                               1
Open Trade PnL                           1.073899
Win Rate [%]                                100.0
Best Trade [%]                          24.758341
Worst Trade [%]                         24.758341
Avg Winning Trade [%

c:\Users\HP Victus 16\AppData\Local\Programs\Python\Python311\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

c:\Users\HP Victus 16\AppData\Local\Programs\Python\Python311\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

c:\Users\HP Victus 16\AppData\Local\Programs\Python\Python311\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

c:\Users\HP Victus 16\AppData\Local\Programs\Python\Python311\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



c:\Users\HP Victus 16\AppData\Local\Programs\Python\Python311\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

c:\Users\HP Victus 16\AppData\Local\Programs\Python\Python311\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

c:\Users\HP Victus 16\AppData\Local\Programs\Python\Python311\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

c:\Users\HP Victus 16\AppData\Local\Programs\Python\Python311\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Processing pair: S50U0^2 and S50Z0^2
Performance stats for S50U0^2 and S50Z0^2:
Start                                         NaT
End                           2020-09-29 00:00:00
Period                                        183
Start Value                                 100.0
End Value                              144.859582
Total Return [%]                        44.859582
Benchmark Return [%]                   -25.416667
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                        14.500573
Max Drawdown Duration                        81.0
Total Trades                                   27
Total Closed Trades                            26
Total Open Trades                               1
Open Trade PnL                          -6.186187
Win Rate [%]                            57.692308
Best Trade [%]                          27.503265
Worst Trade [%]                         -7.254768
Avg Winning Trade [%

Processing pair: S50H1^2 and S50M1^2
Performance stats for S50H1^2 and S50M1^2:
Start                                         NaT
End                           2021-03-30 00:00:00
Period                                        183
Start Value                                 100.0
End Value                                83.09744
Total Return [%]                        -16.90256
Benchmark Return [%]                    12.157586
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                        29.636563
Max Drawdown Duration                       123.0
Total Trades                                   24
Total Closed Trades                            23
Total Open Trades                               1
Open Trade PnL                                0.0
Win Rate [%]                            52.173913
Best Trade [%]                            7.04955
Worst Trade [%]                        -17.304646
Avg Winning Trade [%

...

Processing pair: S50H4^2 and S50M4^2
Performance stats for S50H4^2 and S50M4^2:


c:\Users\HP Victus 16\AppData\Local\Programs\Python\Python311\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

c:\Users\HP Victus 16\AppData\Local\Programs\Python\Python311\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

c:\Users\HP Victus 16\AppData\Local\Programs\Python\Python311\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

c:\Users\HP Victus 16\AppData\Local\Programs\Python\Python311\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                                         NaT
End                           2024-03-28 00:00:00
Period                                        187
Start Value                                 100.0
End Value                              100.929361
Total Return [%]                         0.929361
Benchmark Return [%]                    -7.059476
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                        10.389266
Max Drawdown Duration                       135.0
Total Trades                                    9
Total Closed Trades                             8
Total Open Trades                               1
Open Trade PnL                           0.134601
Win Rate [%]                                 50.0
Best Trade [%]                           4.474506
Worst Trade [%]                         -7.479692
Avg Winning Trade [%]                    2.812933
Avg Losing Trade [%]                    -2.491909


Processing pair: S50U4 and S50Z4
Performance stats for S50U4 and S50Z4:
Start                                         NaT
End                           2024-07-19 00:00:00
Period                                        138
Start Value                                 100.0
End Value                              101.254838
Total Return [%]                         1.254838
Benchmark Return [%]                    -5.387283
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                         6.498027
Max Drawdown Duration                        37.0
Total Trades                                    7
Total Closed Trades                             6
Total Open Trades                               1
Open Trade PnL                          -1.700795
Win Rate [%]                            83.333333
Best Trade [%]                           2.632532
Worst Trade [%]                         -3.696172
Avg Winning Trade [%]       

c:\Users\HP Victus 16\AppData\Local\Programs\Python\Python311\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

c:\Users\HP Victus 16\AppData\Local\Programs\Python\Python311\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

c:\Users\HP Victus 16\AppData\Local\Programs\Python\Python311\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

c:\Users\HP Victus 16\AppData\Local\Programs\Python\Python311\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set

